# Jeff Shum - Technical Case Study - 2020 Churn Analysis

In [1]:
import os
import pandas as pd
import altair as alt
import datetime
fm = pd.read_csv(os.path.join('test-datasets', 'financial_movements.csv'), parse_dates=['financial_movement_date'])
pu = pd.read_csv(os.path.join('test-datasets', 'product_usage.csv'), parse_dates=['date'])
csm = pd.read_csv(os.path.join('test-datasets', 'csm_contacts.csv'), parse_dates=['csm_contact_date'])
nps = pd.read_csv(os.path.join('test-datasets', 'nps_answers.csv'), parse_dates=['nps_date'])

In [2]:
churns = fm[fm.movement_type == 'churn']
churns_monthly = churns.resample('M', on='financial_movement_date').count()
churns_monthly['month'] = churns_monthly.index.strftime('%Y %b')
churns_chart = alt.Chart(churns_monthly['2020']).mark_line(color='maroon').encode(
    x=alt.X('month(month):T', axis=alt.Axis(title='2020')),
    y=alt.Y('movement_type', axis=alt.Axis(title='Number of Churns Each Month (Lower = Better)'))
).properties(width=400,height=400)
churns_sum = churns.resample('M', on='financial_movement_date').sum()
churns_sum['month'] = churns_sum.index.strftime('%Y %b')
churns_sum['abs_val'] = churns_sum.mrr.abs()
churns_sum['2020']
churns_sum_chart = alt.Chart(churns_sum['2020']).mark_area(color='maroon').encode(
    x=alt.X('month(month):T', axis=alt.Axis(title='2020')),
    y=alt.Y('abs_val:Q', axis=alt.Axis(title='Total Churn Negative MRR Per Month (Lower = Better)'))
).properties(width=400,height=400)

# Churn

In [3]:
churns_chart | churns_sum_chart

alt.HConcatChart(...)

* we can see that number of churns per month was lowest in Feb. 2020 but the year ended with more monthly churns that it began with
* the revenue lost from those churns fluctuated for most of 2020 but met a sharp increase in Dec. 2020

In [4]:
nps_avg = nps.resample('M', on='nps_date').mean()
nps_avg['month'] = nps_avg.index.strftime('%Y %b')
net_promoter_scores_2020 = alt.Chart(nps_avg['2020']).mark_line().encode(
    x=alt.X('month(month):T', axis=alt.Axis(title='2020')),
    y=alt.Y('nps_answer', axis=alt.Axis(title='Average NPS Score'))
).properties(width=600,height=400)

# Net Promoter Scores Across 2020

In [5]:
net_promoter_scores_2020

alt.Chart(...)

* The average net promoter scores stayed stable throughout the year even while churn was higher than expected
* It could be wise to investigate if there are ways the net promoter score questionnaire could be more reflective of the value that users are getting from the product

In [6]:
pu_avg = pu.resample('M', on='date').mean()
pu_avg['month'] = pu_avg.index.strftime('%Y %b')
ccr = alt.Chart(pu_avg['2020']).mark_line().encode(
    x=alt.X('month(month):T', axis=alt.Axis(title='2020')),
    y=alt.Y('cards_created', axis=alt.Axis(title='Average Cards Created Per Organization Each Month'))
)
em = alt.Chart(pu_avg['2020']).mark_line().encode(
    x=alt.X('month(month):T', axis=alt.Axis(title='2020')),
    y=alt.Y('email_templates_sent', axis=alt.Axis(title='Average Email Templates Sent Per Organization Each Month'))
)
cf = alt.Chart(pu_avg['2020']).mark_line().encode(
    x=alt.X('month(month):T', axis=alt.Axis(title='2020')),
    y=alt.Y('cards_finished', axis=alt.Axis(title='Average Cards Finished Per Organization Each Month'))
)
cco = alt.Chart(pu_avg['2020']).mark_line().encode(
    x=alt.X('month(month):T', axis=alt.Axis(title='2020')),
    y=alt.Y('cards_connected', axis=alt.Axis(title='Average Cards Connected Per Organization Each Month'))
)

# Product Usage

In [7]:
(ccr | cco) & (cf | em)

alt.VConcatChart(...)

* For 3 out of the 4 product usage features, in 2020 users generally began to use them more than at the start of the year
* However, in May 2020, organizations on the whole began to connect cards *much* less
* If connecting cards affects the value an organization's users get from the product then this decreased usage could have affected churn, I would investigate this more

In [8]:
csm_monthly = csm.resample('M', on='csm_contact_date').count()
csm_monthly['month'] = csm_monthly.index.strftime('%Y %b')
csm_chart = alt.Chart(csm_monthly['2020']).mark_bar().encode(
    x=alt.X('month(month):T', axis=alt.Axis(title='2020')),
    y=alt.Y('csm_contact_date', axis=alt.Axis(title='Total CSM Contacts Per Month'))
).properties(width=600,height=400)

# CSM Contacts

In [9]:
csm_chart

alt.Chart(...)

* Get info from the CSMs about the contents of their correspondence with users to get an idea about what features users find most valuable to them
* Investigate the correlation between CSM contacts and user churn
* One thing of special interest is that in Dec. 2020 CSM contacts dipped but revenue churn more than tripled compared to the previous month--this could be an area to look into more

In [10]:
pu_orgs = pu[(pu.date >= '2020') & (pu.date < '2021')].groupby('date').count()
daily_active_orgs = alt.Chart(pu_orgs[pu_orgs.index.dayofweek < 5].reset_index()).mark_line().encode(
    x=alt.X('date:T',axis=alt.Axis(title='2020 (Excluding Saturdays & Sundays)')),
    y=alt.Y('organization_id', axis=alt.Axis(title='Amount of Active Organizations Each Day'))
).properties(width=800,height=600)

# Daily Active Organizations

In [11]:
daily_active_orgs

alt.Chart(...)

* The number of active organizations gradually rose through 2020, but dipped off around the Northern Hemisphere's Winter holidays when less people are working
* This increase in active users could suggest that:
    * The product has a solid core group of customers who were using the products throughout 2020
    * The growing customer base (new customers and expansions) has meant that despite churn being higher than expected, the number of users of the product are continuing to increase  

# Areas for Further Investigation

* Analyze the behavior of individual users rather than organizations as a whole
    * Consideration of letting your team members see how users interact with the product step-by-step
* Talk to CSM teams to see what customers find most value in
* Try to find ways to increase the daily active users of the product to decrease retention
* Continue to collect data and monitor trends in customer retention
* Create cohort analyses, determine customer lifetime values, conduct 'Survival Analysis'